### Построим модель по данным платформы **KICKSTARTER**, которая будет предсказывать, будет ли выложенный на платформу проект успешным

Импортируем библиотеки и считываем данные

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/ks.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data['Состояние'].value_counts()

Оставим только те состояния, которые соответствуют законченным проектам, то есть `failed` и `successful`

In [ ]:
data = data[data['Состояние'].isin(['failed', 'successful'])]

In [ ]:
data = data.drop('Состояние', axis=1)

Будем предсказывать собранное количество денег, применять модель, а потом уже смотреть, нужная ли сумма получилось. Тогда мы решаем задачу регрессии.

In [ ]:
### Создадим вещественную колонку таргет
data = data.rename({'Собрано в долларах':'таргет'}, axis=1)

Создадим новый признак *Срок*, который будет показывать разницу в днях между датой публикации и дедлайном сбора средств

In [ ]:
### Оставим от колонок с датами разницу в днях 

data['Дедлайн'] = pd.to_datetime(data['Дедлайн'])
data['Дата публикации'] = pd.to_datetime(data['Дата публикации'])

In [ ]:
data['Срок'] = (data['Дедлайн'] - data['Дата публикации']).dt.days

В качестве характеристики положения проекта во времени оставим *Год публикации*

In [ ]:
### Выделим год запуска проекта
data['Год публикации'] = data['Дата публикации'].dt.year

Добавим котировки на каждую дату в наш датасет

In [ ]:
Macro = pd.read_excel("data/macrofeatures.xlsx", engine="openpyxl")

Macro.head()

In [ ]:
### Смерджим колонки. Оставим от макрофичей какую-нибудь одну.

Macro = Macro[['Close_brent', 'dlk_cob_date']].drop_duplicates()

In [ ]:
Macro['dlk_cob_date'] = pd.to_datetime(Macro['dlk_cob_date'])

In [ ]:
data['Дата публикации'] = data['Дата публикации'].dt.date.astype('datetime64[ns]')

In [ ]:
data = pd.merge(data,
         Macro,
         left_on=['Дата публикации'],
         right_on=['dlk_cob_date'],
         how='left')

In [ ]:
data

Получилось много пропущенных значений, заполним их предыдущими значениями

In [ ]:
data = data.sort_values('Дата публикации')

In [ ]:
data['Close_brent'] = data['Close_brent'].fillna(method='ffill')

Самые первые отсутствующие значения заполним первым неотсутствующим

In [ ]:
data['Close_brent'].dropna()

In [ ]:
# берем 34.41
data['Close_brent'] = data['Close_brent'].fillna(34.41)

Удалим лишние колонки

In [ ]:
data = data.drop(['Дедлайн', 'Дата публикации', 'dlk_cob_date'], axis=1)

In [ ]:
data.head()

Удалим также колонки, не являющиеся признаками. 
- *Название* - много уникальных значений, является по сути и индексом, сложно/невозможно установить связь между названием проекта и таргетами
- *Страна* - оставим колонку *Валюта*, которая дублирует информацию
- *Инвесторов* - этот признак формируется одновременно с таргетами, в момент публикации у любого проекта количество инвесторов - 0

In [ ]:
data = data.drop(['Название', 'Страна', 'Инвесторов'], axis=1)

In [ ]:
data.head()

Обработаем категориальные признаки. Применим One-hot encoding к колонкам *Валюта* и *Главная категория*


In [ ]:
data = pd.concat((data, pd.get_dummies(data['Валюта'])), axis=1)
data = data.drop(['Валюта'], axis=1)

In [ ]:
data = pd.concat((data, pd.get_dummies(data['Главная категория'])), axis=1)
data = data.drop(['Главная категория'], axis=1)

In [ ]:
### Почистим данные от лишней информации 
data = data.drop('AUD', axis=1)
data = data.drop('Games', axis=1)

In [ ]:
data.head()

In [ ]:
len(data['Категория'].unique())

Признак *Категория* имеет очень много различных значений. Используем mean-target encoding

In [ ]:
data['Категория'] = data['Категория'].map(data.groupby(['Категория'])['таргет'].mean())

In [ ]:
data.head()

Сохраним подготовленные данные

In [ ]:
data.to_csv('output/processed_data.csv', sep=';')